In [44]:
# #install packages
# !pip3 install requests
# !pip3 install bs4

In [5]:
#import packages
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [26]:
#create lists to store data scraped from web
authors = []
dates = []
statements = []
sources = []
targets = []

In [27]:
#create a function to scrape data from politifact website
#there are 753 pages in the website to scrape
#each page has 30 news items

def scrape_website(page_number):
    page_num = str(page_number)
    URL= 'https://www.politifact.com/factchecks/list/?page='+page_num
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text,"html.parser")
    statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) #Get the tag and it's class
    statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'})  #Get the tag and it's class
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})#Get the tag and it's class
    target = soup.find_all('div', attrs={'class':'m-statement__meter'}) #Get the tag and it's class
   
    #loop through the footer class m-statement__footer to get the date and author
    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name+' '+last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date = month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)

    #Loop through the div m-statement__quote to get the link
    for i in statement_quote:
        link2 = i.find_all('a')
        statements.append(link2[0].text.strip())

    #Loop through the div m-statement__meta to get the source
    for i in statement_meta:
        link3 = i.find_all('a') #Source
        source_text = link3[0].text.strip()
        sources.append(source_text)
        
    #Loop through the target or the div m-statement__meter to get the facts about the statement (True or False)
    for i in target:
        fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(fact)



In [28]:
#Loop through 'n-1' webpages to scrape the data
n=10
for i in range(1, n):
  scrape_website(i)

In [29]:
#Create a new dataFrame 

data = pd.DataFrame(columns = ['author',  'statement', 'source', 'date', 'target']) 
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets

In [31]:
#show dataset
data.head()

,author,statement,source,date,target
0,Gabrielle Settles,The 2017 Unite the Right rally in Charlottesvi...,Scott Adams,"March 10, 2023",pants-fire
1,Madison Czopek,Video shows that certain drinks can “test posi...,Facebook posts,"March 10, 2023",false
2,Paul Specht,Wake County is the only large county in North ...,Erin Paré,"March 10, 2023",mostly-true
3,Ciara O'Rourke,The White House Visitor Center is displaying a...,Facebook posts,"March 9, 2023",pants-fire
4,Ciara O'Rourke,President Joe Biden is “in Gitmo.”,Facebook posts,"March 9, 2023",pants-fire


In [30]:
#get the dimensions of dataset
print(data.shape)

(270, 5)


In [33]:
#get target values
data.target.unique()

array(['pants-fire', 'false', 'mostly-true', 'barely-true', 'true',
       'half-true', 'full-flop', 'half-flip'], dtype=object)

In [35]:
#get the number of news items that are pants-fire or false
print(data.target.value_counts()['pants-fire'])
print(data.target.value_counts()['false'])

46
160


In [36]:
#we are only onterested in pants-fire and false news articles to create our fake news dataset
fake = ['pants-fire','false']
fake_df = data[data['target'].isin(fake)]
fake_df.shape

(206, 5)

In [37]:
#show dataset
data

,author,statement,source,date,target
0,Gabrielle Settles,The 2017 Unite the Right rally in Charlottesvi...,Scott Adams,"March 10, 2023",pants-fire
1,Madison Czopek,Video shows that certain drinks can “test posi...,Facebook posts,"March 10, 2023",false
2,Paul Specht,Wake County is the only large county in North ...,Erin Paré,"March 10, 2023",mostly-true
3,Ciara O'Rourke,The White House Visitor Center is displaying a...,Facebook posts,"March 9, 2023",pants-fire
4,Ciara O'Rourke,President Joe Biden is “in Gitmo.”,Facebook posts,"March 9, 2023",pants-fire
...,...,...,...,...,...
265,Madison Czopek,Buffalo Bills player Damar Hamlin’s collapse w...,Instagram posts,"January 4, 2023",false
266,Yacob Reyes,"After Joe Biden became vice president, at the ...",Joe Biden,"December 20, 2022",false
267,Tom Kertscher,"Says he has been to “Afghanistan, Iraq and tho...",Joe Biden,"December 20, 2022",false
268,Andy Nguyen,“Literally five (homeless) people a day die on...,Karen Bass,"December 20, 2022",mostly-true


In [43]:
#Store the results of scraping first 9 pages to a CSV file
data.to_csv('fake_news1.csv')